In [0]:
dbutils.widgets.removeAll()

In [0]:
# Widgets Setup

# Account ID -- Found in Account Console
dbutils.widgets.text(
  name='account_id',
  defaultValue='',
  label='Account ID'
)

# Which cloud you're on
dbutils.widgets.text(
  name='cloud',
  defaultValue='AWS/Azure/GCP',
  label='Cloud'
)

# State Keyword - Used for security -- ie. ILoveDBX
dbutils.widgets.text(
  name='state',
  defaultValue='ILoveDBX',
  label='State Keyword'
)

In [0]:
# Imports
import uuid
import hashlib
import base64
import requests
import json

# Setup
account_id = dbutils.widgets.get('account_id')

cloud = dbutils.widgets.get('cloud')

state = dbutils.widgets.get("state")

In [0]:
# Change the base URL depending on the cloud
if cloud.lower() == "aws":
  cloud_url = "https://accounts.cloud.databricks.com"
elif cloud.lower() == "azure":
  cloud_url = "https://accounts.azuredatabricks.net"
elif cloud.lower() == "gcp":
  cloud_url = "https://accounts.gcp.databricks.com"
else:
  pass

In [0]:
def generate_codes():
  """
  Generate a code verifier and code challenge for the authorization code flow.
  """

  # Generate a UUID.
  uuid1 = uuid.uuid4()

  # Convert the UUID to a string.
  uuid_str1 = str(uuid1).upper()

  # Create the code verifier.
  code_verifier = uuid_str1 + "-" + uuid_str1

  # Create the code challenge based on the code verifier.
  code_challenge = base64.urlsafe_b64encode(hashlib.sha256(code_verifier.encode()).digest()).decode('utf-8')

  # Remove all padding from the code challenge.
  code_challenge = code_challenge.replace('=', '')

  return code_verifier, code_challenge


# Get & print codes
code_verifier, code_challenge = generate_codes()

print(f"code_verifier: {code_verifier}")
print(f"code_challenge: {code_challenge}")

# Important: Open the below URL in a web browser, then copy the Authorization Code in the URL 
# For example, if the address is http://localhost:8020/?code=dcod...7fe6&state=<state>, the Authorization Code would be dcod...7fe6
print(f"{cloud_url}/oidc/accounts/{account_id}/v1/authorize?client_id=databricks-cli&redirect_url=localhost:8020&response_type=code&state={state}&code_challenge={code_challenge}&code_challenge_method=S256&scope=all-apis+offline_access"  )

In [0]:

authorization_code = ""  # Paste Authorization Code here

# Submit call to the Databricks Account API to create an access token
url = f"{cloud_url}/oidc/accounts/{account_id}/v1/token"
data = {
    "client_id": "databricks-cli",
    "grant_type": "authorization_code",
    "scope": "all-apis offline_access",
    "redirect_uri": "http://localhost:8020",
    "code_verifier": code_verifier,
    "code": authorization_code
}

response = requests.post(url, data=data)

# Parse out response
print(response.status_code)
response_json = response.json()
print(response_json)

In [0]:
# Parse out response
account_access_token = response_json['access_token']
print(account_access_token)

In [0]:
# Example of how to use the AccountClient to list workspaces in a Databricks Account

from databricks.sdk import AccountClient

client = AccountClient(
    token=account_access_token,
    account_id=account_id,
    host=cloud_url
)
 
for w in client.workspaces.list():
    print(w.workspace_id, w.workspace_name)